In [4]:
!pip install sentence-transformers


In [16]:
# 📌 Step 2: Import Libraries
import psycopg2
import pandas as pd
import os

# 📌 Step 3: Set Up Database Connection (Replace with Your Details)
DATABASE_URL = "dbname='postgres' user='postgres' host='34.59.119.208' password='avantichhaya'"

def connect_db():
    try:
        conn = psycopg2.connect(DATABASE_URL)
        print("✅ Connected to PostgreSQL Cloud SQL")
        return conn
    except Exception as e:
        print("🚨 Connection Error:", e)
        return None

conn = connect_db()
cursor = conn.cursor()


✅ Connected to PostgreSQL Cloud SQL


In [17]:
from sentence_transformers import SentenceTransformer
import numpy as np

encoder = SentenceTransformer('all-mpnet-base-v2')

cursor.execute("""
    SELECT email_id, subject, body, classified_category
    FROM emails
    WHERE escalated = TRUE
      AND learn = TRUE
      AND classified_category IS NOT NULL
      AND classified_category != 'human_intervention'
      AND email_id NOT IN (
          SELECT email_id FROM learned_email_vectors
      );
""")

email_rows = cursor.fetchall()

if email_rows:
    for email_id, subject, body, category in email_rows:

        text = f"Subject: {subject}\nBody: {body}"
        vector = encoder.encode(text)
        vector_list = vector.tolist()

        insert_query = """
        INSERT INTO learned_email_vectors (email_id, category, vector)
        VALUES (%s, %s, %s)
        ON CONFLICT (email_id) DO NOTHING;
        """

        cursor.execute(insert_query, (email_id, category, vector_list))
        conn.commit()

        print(f"✅ Vector inserted for email_id {email_id} with category '{category}'.")
else:
    print("⚠️ No eligible emails found. Nothing to insert.")


✅ Vector inserted for email_id 139 with category 'Recruitment Process'.


In [18]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM learned_email_vectors;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_48268\2096386738.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM learned_email_vectors;", conn)


,email_id,category,vector,created_at
0,140,Onboarding,"[0.0015864443266764283, -0.0223986878991127, -...",2025-05-05 00:48:08.396391
1,137,Recruitment Process,"[0.0169376228004694, -0.01755315624177456, -0....",2025-05-05 00:51:49.256918
2,112,Recruitment Process,"[0.03189278766512871, -0.05429130792617798, 0....",2025-05-05 03:24:23.475210
3,139,Recruitment Process,"[0.010934802703559399, 0.01625041849911213, -0...",2025-05-05 14:44:21.656742
